In [27]:
# the Python Requests package will allow us to send HTTP requests to get HTML files
import requests

# the GET method indicates that you’re trying to get or retrieve data from a specified resource. 
# to make a GET request, invoke requests.get()
from requests import get

# Beautiful Soup is a Python library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

# pandas is a Python data analysis library
import pandas as pd

# NumPy is a Python library used for working with large, multi-dimensional arrays and matrices
import numpy as np

from datetime import datetime

In [29]:
url = 'https://speedgreens.co/product-category/'

keys = ['concentrates/hash/', 'cannabis/sativa/']

all_names = []

all_prices = []

all_quantities = []

date = []

for key in keys:
    results = requests.get(url + key)

    soup = BeautifulSoup(results.content, "html.parser")

    products = soup.find_all('div', class_="product-content")

    for product in products:
        product_link = product.find('a', class_= 'product-loop-title')
        name = product_link.text
        product_page = product_link.get('href')

        product_page_results = requests.get(product_page)
        product_page_soup = BeautifulSoup(product_page_results.content, "html.parser")
        
        price = product_page_soup.find('span', class_='woocommerce-Price-amount amount').text

        quantity = product_page_soup.find('a', class_='filter-item').text
        
        current_month = datetime.now().month
        current_year = datetime.now().year

        all_names.append(name)
        all_prices.append(price)
        all_quantities.append(quantity)
        date.append(str(current_month) + '/' + str(current_year))
        
        print(name)
        print(product_page)
        print('Price: ' + price)
        print('Unit: ' + quantity)



Two Sword Hash (AA) – Hash Plant 
https://speedgreens.co/product/two-sword-hash-aa/
Price: $8.00
Unit: 1gm

Rockstar Tuna Bubble Hash (AAAA) 
https://speedgreens.co/product/tuna-rockstar-craft-hash-aaaa/
Price: $22.00
Unit: 1gm

Pink Gas Premium Craft Bubble Hash (AAAA) – Hash Weed 
https://speedgreens.co/product/pink-gas-craft-hash-aaaa/
Price: $22.00
Unit: 1gm

Tesla Hash (AAA) – Hash Weed 
https://speedgreens.co/product/tesla-hash-aaa/
Price: $14.00
Unit: 1gm

Diamond Hash (AAA) – Hash Weed 
https://speedgreens.co/product/diamond-hash/
Price: $14.00
Unit: 1gm

Mercedes Hash (AA) – Hashish 
https://speedgreens.co/product/mercedes-hash-aa/
Price: $8.00
Unit: 1gm

Zigzag Hash (AAA) – Hashish 
https://speedgreens.co/product/zig-zag-hash-aaa/
Price: $14.00
Unit: 1gm

Afghan Mazar Sharif Hash (AAAA) – Hashish 
https://speedgreens.co/product/mazar-sharif-afghan-hash-aaaa/
Price: $22.00
Unit: 1gm

Nepalese Temple Ball Hash (AAAA) – Nepal Hash 
https://speedgreens.co/product/nepalese-temple

In [32]:
df = pd.DataFrame(
    {'Date': date,
     'Product': all_names,
     'Price': all_prices,
     'Quantity': all_quantities
    })

In [33]:
df

,Date,Product,Price,Quantity
0,2/2022,\nTwo Sword Hash (AA) – Hash Plant,$8.00,1gm
1,2/2022,\nRockstar Tuna Bubble Hash (AAAA),$22.00,1gm
2,2/2022,\nPink Gas Premium Craft Bubble Hash (AAAA) – ...,$22.00,1gm
3,2/2022,\nTesla Hash (AAA) – Hash Weed,$14.00,1gm
4,2/2022,\nDiamond Hash (AAA) – Hash Weed,$14.00,1gm
5,2/2022,\nMercedes Hash (AA) – Hashish,$8.00,1gm
6,2/2022,\nZigzag Hash (AAA) – Hashish,$14.00,1gm
7,2/2022,\nAfghan Mazar Sharif Hash (AAAA) – Hashish,$22.00,1gm
8,2/2022,\nNepalese Temple Ball Hash (AAAA) – Nepal Hash,$20.00,1gm
9,2/2022,\nIWC Moroccan Hash (AAAA) – Hashish,$25.00,1gm


In [34]:
# get script to run every month?
# add date  as column
# append to a csv file with previous data every time script is run

df.to_csv('speedgreens.csv')